##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Генерация текста с RNN

Блокнот с демонстрацией размеров бедствия.

This tutorial demonstrates how to generate text using a character-based RNN. See Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
ROMEO:
(Зходит и тебегойнный была в зиму). (Мне кровитыл… м-до порумночить первити, засиласт? Я — не умля…

Актер:
Кораз, бывеша-пил!

Пепел:
Ну, но пониеть? Обобыват жилить мне ему выпра вет!

Сатин:
Я что — нудушка…

Бубнов:
Я-чу, Бубном… выт кожется вудаешь, илешше на… совотет в еще… чму ты иво… 5?

Медава.
На доваю… и газобя… не прадеет базопрюну! ПрТавдаре, что это я — на почастра пляку пряя…

Кустылев:
А ты…

Лука:
Придут ил…

Квавнов:
(удах рамивыл а нак, говорит… И всох! Звазаритилей, я тосиры — убедного невол дуг ух жий и босторо…

Актер:
Жу лебил гогобе! Гдо-хо? 
Актер:
Ид-тро довжень

Анна:
Он бы, мно все угошя» говерится паркасьсь…

Пепел:
(талко свызнываясь, Клащ путьитьны подиги, потвнопет, Бароннны, куса и холеши, На нает — всю и с чела.) (Аморы окожнья рузнай… порте моя! Ну най… дары человек?

Пепел:
Она — звалае сможет! А ди — погодет ни ондевет?

Бубнов:
Вери сленается!

Наска:
Ну, видох! Уйлищее это? Ну, долодное быесту…

Пепе 

</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [3]:
path_to_file = tf.keras.utils.get_file('Gorky-Na_dne_ru.txt', 'https://raw.githubusercontent.com/AlexandrParkhomenko/ai/main/cs25/Gorky-Na_dne_ru.txt')

189684/189684 [==============================] - 0s 1us/step


### Read the data

First, look in the text:

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 107947 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

На дне

Действующие лица

Посвящаю
Константину Петровичу Пятницкому

Михаил Иванов Костылев, 54 года, содержатель ночлежки.
Василиса Карповна, его жена, 26 лет.
Наташа, ее сестра, 20 лет.
Медведев, их дядя, полицейский, 50 лет.
Васька Пепе


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

92 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [7]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

2022-08-18 22:38:55.428652: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-18 22:38:57.762124: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4447 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5
2022-08-18 22:38:57.765428: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[0, 0, 24, 0, 25, 0, 0], [0, 0, 0]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'[UNK]', b'[UNK]', b'c', b'[UNK]', b'e', b'[UNK]', b'[UNK]'],
 [b'[UNK]', b'[UNK]', b'[UNK]']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'[UNK][UNK]c[UNK]e[UNK][UNK]', b'[UNK][UNK][UNK]'], dtype=object)

In [13]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(107947,), dtype=int64, numpy=array([43, 58,  3, ...,  1,  2,  1])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

Н
а
 
д
н
е








In [17]:
seq_length = 100


The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [18]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\xd0\x9d' b'\xd0\xb0' b' ' b'\xd0\xb4' b'\xd0\xbd' b'\xd0\xb5' b'\r'
 b'\n' b'\r' b'\n' b'\xd0\x94' b'\xd0\xb5' b'\xd0\xb9' b'\xd1\x81'
 b'\xd1\x82' b'\xd0\xb2' b'\xd1\x83' b'\xd1\x8e' b'\xd1\x89' b'\xd0\xb8'
 b'\xd0\xb5' b' ' b'\xd0\xbb' b'\xd0\xb8' b'\xd1\x86' b'\xd0\xb0' b'\r'
 b'\n' b'\r' b'\n' b'\xd0\x9f' b'\xd0\xbe' b'\xd1\x81' b'\xd0\xb2'
 b'\xd1\x8f' b'\xd1\x89' b'\xd0\xb0' b'\xd1\x8e' b'\r' b'\n' b'\xd0\x9a'
 b'\xd0\xbe' b'\xd0\xbd' b'\xd1\x81' b'\xd1\x82' b'\xd0\xb0' b'\xd0\xbd'
 b'\xd1\x82' b'\xd0\xb8' b'\xd0\xbd' b'\xd1\x83' b' ' b'\xd0\x9f'
 b'\xd0\xb5' b'\xd1\x82' b'\xd1\x80' b'\xd0\xbe' b'\xd0\xb2' b'\xd0\xb8'
 b'\xd1\x87' b'\xd1\x83' b' ' b'\xd0\x9f' b'\xd1\x8f' b'\xd1\x82'
 b'\xd0\xbd' b'\xd0\xb8' b'\xd1\x86' b'\xd0\xba' b'\xd0\xbe' b'\xd0\xbc'
 b'\xd1\x83' b'\r' b'\n' b'\r' b'\n' b'\xd0\x9c' b'\xd0\xb8' b'\xd1\x85'
 b'\xd0\xb0' b'\xd0\xb8' b'\xd0\xbb' b' ' b'\xd0\x98' b'\xd0\xb2'
 b'\xd0\xb0' b'\xd0\xbd' b'\xd0\xbe' b'\xd0\xb2' b' ' b'\xd0\x9a'
 b'\xd0\x

It's easier to see what this is doing if you join the tokens back into strings:

In [19]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\xd0\x9d\xd0\xb0 \xd0\xb4\xd0\xbd\xd0\xb5\r\n\r\n\xd0\x94\xd0\xb5\xd0\xb9\xd1\x81\xd1\x82\xd0\xb2\xd1\x83\xd1\x8e\xd1\x89\xd0\xb8\xd0\xb5 \xd0\xbb\xd0\xb8\xd1\x86\xd0\xb0\r\n\r\n\xd0\x9f\xd0\xbe\xd1\x81\xd0\xb2\xd1\x8f\xd1\x89\xd0\xb0\xd1\x8e\r\n\xd0\x9a\xd0\xbe\xd0\xbd\xd1\x81\xd1\x82\xd0\xb0\xd0\xbd\xd1\x82\xd0\xb8\xd0\xbd\xd1\x83 \xd0\x9f\xd0\xb5\xd1\x82\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb8\xd1\x87\xd1\x83 \xd0\x9f\xd1\x8f\xd1\x82\xd0\xbd\xd0\xb8\xd1\x86\xd0\xba\xd0\xbe\xd0\xbc\xd1\x83\r\n\r\n\xd0\x9c\xd0\xb8\xd1\x85\xd0\xb0\xd0\xb8\xd0\xbb \xd0\x98\xd0\xb2\xd0\xb0\xd0\xbd\xd0\xbe\xd0\xb2 \xd0\x9a\xd0\xbe\xd1\x81\xd1\x82\xd1\x8b\xd0\xbb\xd0\xb5\xd0\xb2, 5'
b'4 \xd0\xb3\xd0\xbe\xd0\xb4\xd0\xb0, \xd1\x81\xd0\xbe\xd0\xb4\xd0\xb5\xd1\x80\xd0\xb6\xd0\xb0\xd1\x82\xd0\xb5\xd0\xbb\xd1\x8c \xd0\xbd\xd0\xbe\xd1\x87\xd0\xbb\xd0\xb5\xd0\xb6\xd0\xba\xd0\xb8.\r\n\xd0\x92\xd0\xb0\xd1\x81\xd0\xb8\xd0\xbb\xd0\xb8\xd1\x81\xd0\xb0 \xd0\x9a\xd0\xb0\xd1\x80\xd0\xbf\xd0\xbe\xd0\xb2\xd0\xbd\xd0\xb0, \xd0\xb5

For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\xd0\x9d\xd0\xb0 \xd0\xb4\xd0\xbd\xd0\xb5\r\n\r\n\xd0\x94\xd0\xb5\xd0\xb9\xd1\x81\xd1\x82\xd0\xb2\xd1\x83\xd1\x8e\xd1\x89\xd0\xb8\xd0\xb5 \xd0\xbb\xd0\xb8\xd1\x86\xd0\xb0\r\n\r\n\xd0\x9f\xd0\xbe\xd1\x81\xd0\xb2\xd1\x8f\xd1\x89\xd0\xb0\xd1\x8e\r\n\xd0\x9a\xd0\xbe\xd0\xbd\xd1\x81\xd1\x82\xd0\xb0\xd0\xbd\xd1\x82\xd0\xb8\xd0\xbd\xd1\x83 \xd0\x9f\xd0\xb5\xd1\x82\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb8\xd1\x87\xd1\x83 \xd0\x9f\xd1\x8f\xd1\x82\xd0\xbd\xd0\xb8\xd1\x86\xd0\xba\xd0\xbe\xd0\xbc\xd1\x83\r\n\r\n\xd0\x9c\xd0\xb8\xd1\x85\xd0\xb0\xd0\xb8\xd0\xbb \xd0\x98\xd0\xb2\xd0\xb0\xd0\xbd\xd0\xbe\xd0\xb2 \xd0\x9a\xd0\xbe\xd1\x81\xd1\x82\xd1\x8b\xd0\xbb\xd0\xb5\xd0\xb2, '
Target: b'\xd0\xb0 \xd0\xb4\xd0\xbd\xd0\xb5\r\n\r\n\xd0\x94\xd0\xb5\xd0\xb9\xd1\x81\xd1\x82\xd0\xb2\xd1\x83\xd1\x8e\xd1\x89\xd0\xb8\xd0\xb5 \xd0\xbb\xd0\xb8\xd1\x86\xd0\xb0\r\n\r\n\xd0\x9f\xd0\xbe\xd1\x81\xd0\xb2\xd1\x8f\xd1\x89\xd0\xb0\xd1\x8e\r\n\xd0\x9a\xd0\xbe\xd0\xbd\xd1\x81\xd1\x82\xd0\xb0\xd0\xbd\xd1\x82\xd0\xb8\xd0\xbd

### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [25]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [27]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://www.tensorflow.org/static/text/tutorials/images/text_generation_training.png)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

2022-08-18 22:39:47.570912: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


(64, 100, 93) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  23808     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  95325     
                                                                 
Total params: 4,057,437
Trainable params: 4,057,437
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [31]:
sampled_indices

array([53, 38, 71, 66, 82, 74, 55, 63, 92, 22, 68, 65, 75, 69, 51, 59, 72,
       86, 34, 41, 50, 36, 27,  7, 75, 19, 56,  5, 88, 41, 77, 50, 76, 64,
       25, 61, 44, 13, 45, 27, 63, 77, 35, 61, 88, 39, 69, 34, 80, 41, 81,
       38, 62, 31, 64, 60, 20, 50, 29, 46, 69, 14, 29,  4, 69,  5, 10, 72,
       39, 78, 21, 41, 13, 92, 28, 81, 29, 62, 55, 56,  7, 59, 22, 14, 43,
       80, 26, 49, 24, 28, 75, 79, 43, 47, 62, 72, 21, 46, 49, 69])

Decode these to see the text predicted by this untrained model:

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'\xd0\xb0\xd0\xb1\xd0\xb0\xd0\xba\xd0\xb5 \xd0\xbf\xd1\x80\xd0\xbe\xd0\xbf\xd1\x8c\xd0\xb5\xd1\x82\xd0\xb5\xe2\x80\xa6 (\xd0\xa1\xd0\xb0\xd0\xb4\xd0\xb8\xd1\x82\xd1\x81\xd1\x8f \xd0\xbd\xd0\xb0 \xd0\xbd\xd0\xb0\xd1\x80\xd1\x8b.) \xd0\xa1\xd1\x82\xd0\xb0\xd1\x80\xd1\x8b\xd0\xb9 \xd1\x87\xd0\xb5\xd1\x80\xd1\x82\xe2\x80\xa6 \xd1\x80\xd0\xb0\xd0\xb7\xd0\xb1\xd1\x83\xd0\xb4\xd0\xb8\xd0\xbb\xe2\x80\xa6 \xd0\x90 \xd1\x8f \xe2\x80\x94 \xd1\x81\xd0\xbe\xd0\xbd \xd1\x85\xd0\xbe\xd1\x80\xd0\xbe\xd1\x88\xd0\xb8\xd0\xb9 \xd0\xb2\xd0\xb8\xd0\xb4\xd0\xb5\xd0\xbb: \xd0\xb1\xd1\x83\xd0\xb4\xd1\x82\xd0\xbe \xd0\xbb\xd0\xbe\xd0\xb2\xd0\xbb\xd1\x8e \xd1\x8f \xd1\x80\xd1\x8b\xd0\xb1'

Next Char Predictions:
 b'\xd0\xa7\xd0\x98\xd0\xbd\xd0\xb8\xd1\x88\xd1\x80\xd0\xab\xd0\xb5\xe2\x80\xa6?\xd0\xba\xd0\xb7\xd1\x81\xd0\xbb\xd0\xa5\xd0\xb1\xd0\xbe\xd1\x8c\xd0\x94\xd0\x9b\xd0\xa4\xd0\x96r)\xd1\x819\xd0\xad"\xd1\x8e\xd0\x9b\xd1\x83\xd0\xa4\xd1\x82\xd0\xb6e\xd0\xb3\xd0\x9e2\xd0\x9fr\xd0\xb5\xd1\x83\xd0\x95

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 93)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.532577, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [35]:
tf.exp(example_batch_mean_loss).numpy()

92.99792

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [36]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [38]:
EPOCHS = 20

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
16/16 [==============================] - 4s 73ms/step - loss: 4.4288
Epoch 2/20
16/16 [==============================] - 1s 72ms/step - loss: 3.7765
Epoch 3/20
16/16 [==============================] - 1s 70ms/step - loss: 3.1476
Epoch 4/20
16/16 [==============================] - 1s 70ms/step - loss: 2.7697
Epoch 5/20
16/16 [==============================] - 1s 71ms/step - loss: 2.5529
Epoch 6/20
16/16 [==============================] - 1s 70ms/step - loss: 2.4456
Epoch 7/20
16/16 [==============================] - 1s 70ms/step - loss: 2.3778
Epoch 8/20
16/16 [==============================] - 1s 71ms/step - loss: 2.3137
Epoch 9/20
16/16 [==============================] - 1s 72ms/step - loss: 2.2581
Epoch 10/20
16/16 [==============================] - 1s 69ms/step - loss: 2.2094
Epoch 11/20
16/16 [==============================] - 1s 71ms/step - loss: 2.1622
Epoch 12/20
16/16 [==============================] - 1s 70ms/step - loss: 2.1208
Epoch 13/20
16/16 [==================

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://www.tensorflow.org/static/text/tutorials/images/text_generation_sampling.png)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [40]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [41]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [42]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
(Зходит и тебегойнный была в зиму). (Мне кровитыл… м-до порумночить первити, засиласт? Я — не умля…

Актер:
Кораз, бывеша-пил!

Пепел:
Ну, но пониеть? Обобыват жилить мне ему выпра вет!

Сатин:
Я что — нудушка…

Бубнов:
Я-чу, Бубном… выт кожется вудаешь, илешше на… совотет в еще… чму ты иво… 5?

Медава.
На доваю… и газобя… не прадеет базопрюну! ПрТавдаре, что это я — на почастра пляку пряя…

Кустылев:
А ты…

Лука:
Придут ил…

Квавнов:
(удах рамивыл а нак, говорит… И всох! Звазаритилей, я тосиры — убедного невол дуг ух жий и босторо…

Актер:
Жу лебил гогобе! Гдо-хо? 
Актер:
Ид-тро довжень

Анна:
Он бы, мно все угошя» говерится паркасьсь…

Пепел:
(талко свызнываясь, Клащ путьитьны подиги, потвнопет, Бароннны, куса и холеши, На нает — всю и с чела.) (Аморы окожнья рузнай… порте моя! Ну най… дары человек?

Пепел:
Она — звалае сможет! А ди — погодет ни ондевет?

Бубнов:
Вери сленается!

Наска:
Ну, видох! Уйлищее это? Ну, долодное быесту…

Пепе 

______________________________________

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [43]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'ROMEO:\r\n\xd0\x9a\xd0\xb0\xd0\xba\xd0\xb8\xd1\x85\xd0\xb0 \xd0\xb8\xd0\xb7\xd0\xb3\xd0\xbe\xd0\xbd\xd1\x8f\xd1\x82\xd0\xb8\xd0\xbb \xd0\x90\xd0\xbd-\xd1\x82\xd0\xbe\xd1\x81\xd0\xbe\xd1\x81 \xd0\xbc\xd0\xb8\xd0\xbd\xd1\x8f\xd1\x87\xd0\xbf\xd0\xbe\xd1\x81\xd0\xb8\xd1\x82 \xd0\xba\xd0\xb0\xd1\x80\xd1\x8b\xd0\xb2\xd1\x8b\xd0\xbb\xd0\xb0\xd1\x8e, \xd0\xbe\xd0\xbd\xd0\xb0\xd0\xb7 \xd0\xbe\xd0\xbb\xd0\xba\xd0\xb0\xd0\xb5\xd1\x82\xd0\xb5\xd0\xb5 \xd0\x92\xd0\xbe\xd1\x81\xd1\x82\xd0\xb8\xd0\xbb \xd1\x81\xd1\x8f\xd0\xb4\xd0\xb8\xd1\x82 \xd0\xbc\xd0\xb8\xd0\xbd\xd1\x8f\xd1\x82, \xd1\x85\xd0\xbe\xd1\x82\xd0\xbe\xd1\x80\xd0\xb8\xd1\x82\xd0\xbe \xe2\x80\x94 \xd0\xbd\xd0\xb5 \xd0\xbc\xd1\x83\xd0\xb6\xd0\xb5\xd1\x82\xd1\x8c? \xd0\xaf\xd0\xba\xd0\xbe, \xd1\x83\xd1\x88\xd0\xbb\xd0\xb0\xd0\xbd\xd1\x8b\xd0\xb2\xd0\xb0\xd1\x81\xd1\x8c! \xd0\xb7\xd0\xb3\xd0\xb0\xd1\x80\xd0\xb8 \xe2\x80\x94 \xd1\x8f \xd0\xbd\xd0\xb5 \xd1\x81\xd0\xbb\xd0\xb5\xd0\xb4\xd0\xb8\xd1\x82\xd1\x8c\xd1\x81\xd1\x8f\xe2\x

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [44]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [45]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:MФЗ»…
(Ах скраци говоруи, воли честь мне люди звапит.)
За мого! Вись? я ос хитела ду — почим! Я то


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [46]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [47]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [48]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [49]:
model.fit(dataset, epochs=1)

16/16 [==============================] - 5s 60ms/step - loss: 4.4260


Or if you need more control, you can write your own complete custom training loop:

In [50]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.1801

Epoch 1 Loss: 3.6501
Time taken for 1 epoch 2.23 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 3.3234

Epoch 2 Loss: 3.1389
Time taken for 1 epoch 1.13 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 2.9096

Epoch 3 Loss: 2.7645
Time taken for 1 epoch 1.09 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 2.6212

Epoch 4 Loss: 2.5562
Time taken for 1 epoch 1.09 sec
________________________________________________________________________________
Epoch 5 Batch 0 Loss 2.4759

Epoch 5 Loss: 2.4472
Time taken for 1 epoch 1.67 sec
________________________________________________________________________________
Epoch 6 Batch 0 Loss 2.4069

Epoch 6 Loss: 2.3768
Time taken for 1 epoch 1.07 sec
________________________________________________________________________________
Epoch 7 Batch 0 Loss 2